In [3]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

import cv2
from ultralytics import YOLO

# -----------------------
# GPU 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"[INFO] {len(gpus)} GPU(s) available: {[gpu.name for gpu in gpus]}")
else:
    print("[INFO] No GPU available. Training will use CPU.")

# -----------------------
# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Seed 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# -----------------------
# 경로 설정
ORIGINAL_TRAIN_DIR = "D:/데이콘 250519 대회/open/train"        # 원본 원천 데이터셋
FILTERED_TRAIN_DIR = "D:/데이콘 250519 대회/filtered_train"    # 필터링 결과 저장 폴더
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# -----------------------
# YOLOv8 차량 외관 필터링 함수

VEHICLE_CLASSES = [2, 5, 7]  # car, bus, truck 클래스 번호

yolo_model = YOLO('yolov8n.pt')  # 필요시 yolov8s.pt 등으로 변경 가능

def is_full_vehicle(detections, img):
    """
    차량 전체 외관 포함 여부 판단 함수.
    트렁크, 보닛 열린 이미지, 내부 이미지 등을 제거하기 위한 조건 강화
    """
    h, w = img.shape[:2]
    img_area = h * w

    full_vehicle_found = False

    for det in detections:
        cls = int(det.cls)
        if cls not in VEHICLE_CLASSES:
            continue

        # 박스 좌표 추출
        x1, y1, x2, y2 = det.xyxy[0].cpu().numpy()
        box_w, box_h = x2 - x1, y2 - y1
        area = box_w * box_h
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        aspect_ratio = box_w / box_h

        # 조건 1: 차량 클래스일 것
        # 조건 2: 박스가 이미지 중앙에 위치 (중앙 근처, 중심 좌표 비율 조정)
        if not (0.4 * w < center_x < 0.6 * w and 0.4 * h < center_y < 0.6 * h):
            continue

        # 조건 3: 박스 면적이 이미지 대비 최소 40% 이상 (작은 박스는 제거)
        if area / img_area < 0.40:
            continue

        # 조건 4: 박스의 비율 제한 (너무 좁거나 넓으면 제외)
        # 보닛, 트렁크 열린 경우 가로세로 비율 극단적일 수 있음
        if aspect_ratio < 0.9 or aspect_ratio > 1.8:
            continue

        # 조건 5: 보닛이나 트렁크 열린 이미지/내부 이미지는 비율, 위치 조건으로 대부분 걸러짐
        # 필요시 추가 조건 작성 가능

        # 위 조건 모두 만족 시 전체 차량 외관 포함으로 판단
        full_vehicle_found = True
        break

    return full_vehicle_found

# -----------------------
# 차량 외관 필터링 수행

print("차량 외관 이상치 필터링 시작...")
os.makedirs(FILTERED_TRAIN_DIR, exist_ok=True)
class_dirs = sorted(os.listdir(ORIGINAL_TRAIN_DIR))

filtered_count = 0
total_count = 0

for cls in tqdm(class_dirs, desc="클래스별 필터링"):
    input_dir = os.path.join(ORIGINAL_TRAIN_DIR, cls)
    output_dir = os.path.join(FILTERED_TRAIN_DIR, cls)
    os.makedirs(output_dir, exist_ok=True)

    image_paths = glob(os.path.join(input_dir, "*.jpg"))
    for img_path in image_paths:
        total_count += 1
        img = cv2.imread(img_path)
        results = yolo_model(img, verbose=False)[0]

        if is_full_vehicle(results.boxes, img):
            filename = os.path.basename(img_path)
            cv2.imwrite(os.path.join(output_dir, filename), img)
            filtered_count += 1

print(f"🔍 전체 이미지 수: {total_count}")
print(f"✅ 필터링된 차량 외관 이미지 수: {filtered_count}")
print(f"🧼 이상치 제거 완료. 정제 비율: {filtered_count / total_count:.2%}")

# -----------------------
# 학습 데이터 준비

label_list = sorted(os.listdir(FILTERED_TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

image_paths = glob(os.path.join(FILTERED_TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# -----------------------
# 데이터 전처리 함수

def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )

    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# -----------------------
# 모델 생성

def build_model(num_classes):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
        weights='imagenet',
        pooling='avg'
    )
    x = layers.Dense(num_classes, activation='softmax')(base.output)
    model = models.Model(inputs=base.input, outputs=x)
    return model

# -----------------------
# Stratified K-Fold 학습

skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1} 시작 ###")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(
        optimizer=optimizers.Adam(CFG['LR']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=CFG['EPOCHS'],
        verbose=1
    )

# -----------------------
# 테스트 데이터 추론 및 제출 파일 생성

test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

print("테스트 데이터 추론 시작...")
preds = model.predict(test_ds)
print("추론 완료")

submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]

submission.to_csv("submission.csv", index=False)
print("submission.csv 저장 완료")


[INFO] 1 GPU(s) available: ['/physical_device:GPU:0']
차량 외관 이상치 필터링 시작...


클래스별 필터링: 100%|██████████| 396/396 [37:04<00:00,  5.62s/it]


🔍 전체 이미지 수: 33137
✅ 필터링된 차량 외관 이미지 수: 20022
🧼 이상치 제거 완료. 정제 비율: 60.42%

### Fold 1 시작 ###
Epoch 1/15
667/667 [==============================] - 285s 207ms/step - loss: 3.4425 - accuracy: 0.3617 - val_loss: 3.5410 - val_accuracy: 0.2576
Epoch 2/15
667/667 [==============================] - 62s 90ms/step - loss: 1.1479 - accuracy: 0.7056 - val_loss: 2.0320 - val_accuracy: 0.4530
Epoch 3/15
667/667 [==============================] - 61s 89ms/step - loss: 0.5029 - accuracy: 0.8662 - val_loss: 1.4907 - val_accuracy: 0.5845
Epoch 4/15
667/667 [==============================] - 61s 89ms/step - loss: 0.2986 - accuracy: 0.9184 - val_loss: 1.0644 - val_accuracy: 0.6972
Epoch 5/15
667/667 [==============================] - 62s 90ms/step - loss: 0.2055 - accuracy: 0.9432 - val_loss: 0.8718 - val_accuracy: 0.7551
Epoch 6/15
667/667 [==============================] - 62s 89ms/step - loss: 0.1543 - accuracy: 0.9568 - val_loss: 0.9210 - val_accuracy: 0.7546
Epoch 7/15
667/667 [========================